This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.7) and depends on Magma (tested using version
2.27-1).

In this notebook, we make an intermediate calculation towards finding generic curves of genus 6 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1: we compute orbit representatives for 3-dimensional flats in the dual of $\mathbf{P}^{9}$.  Allow 40 minutes for completion.

In [1]:
load("preamble.sage")

Construct generators for $\mathrm{GL}(5, \mathbb{F}_2)$ acting on $\wedge^2 \mathbb{F}_2^5$.

In [2]:
F = GF(2)

In [3]:
G0 = SymmetricGroup(10)
g1 = G0([(1,5,8,10,4),(2,6,9,3,7)]).matrix().change_ring(F)
g2 = identity_matrix(F, 10)
g2[1,4] = 1
g2[2,5] = 1
g2[3,6] = 1
G = MatrixGroup([g1,g2])

Construct an orbit lookup tree of depth 4 for the action of $G$ on the dual of $\mathbf{P}^9$.

In [4]:
def apply_group_elem(g, x):
    x1 = g*x
    x1.set_immutable()
    return x1

In [5]:
def stabilizer(x):
    return vec_stab(Matrix(F, x), transpose=True)

In [6]:
def optimized_rep(g):
    return g.matrix()

In [7]:
def forbid(vecs, easy=False):
    return (Matrix(vecs).rank() < len(vecs))

In [8]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep,
           'forbid': forbid}

In [9]:
S = [x for x in VectorSpace(F, 10) if x != 0]
for x in S:
    x.set_immutable()

In [10]:
tree = build_orbit_tree(G, S, 4, methods, verbose=False)
len(green_nodes(tree, 4))

6306

Construct generators of the group $\mathrm{GL}(4, \mathbb{F}_2)$.

In [11]:
H = GL(4,F)
gens = []
while H.subgroup(gens).order() != H.order():
    gens.append(H.random_element().matrix())
len(gens)

3

Use the action of $\mathrm{GL}(4, \mathbb{F}_2)$ to identify 4-tuples whose spans are $G$-equivalent.

In [12]:
edges = []
for mats in green_nodes(tree, 4):
    for g in gens:
        mats0 = tuple(sum(g[i,j]*mats[j] for j in range(4)) for i in range(4))
        for v in mats0:
            v.set_immutable()
        mats1, g1 = orbit_rep_from_tree(G, tree, mats0, apply_group_elem, optimized_rep)
        edges.append((mats, mats1))

In [13]:
Gamma = Graph(edges, loops=True)
l = [cc[0] for cc in Gamma.connected_components(sort=False)]
len(l)

57

Write coordinates of representative tuples to a file.

In [14]:
with open("genus6-flats.txt", "w") as f:
    f.write(str(l))